In [2]:
#Tienes que poner el spawn proteccion en 0 para que puedas jugar de manera normal(esto dentro de Server.properties)
#importa librerias necesarias para ejecucion

import requests

import time

import os

import re

import json

import glob

import threading

import subprocess


#Actualizacion de lista de paquetes

!sudo apt update &>/dev/null && echo "el apt cache fue actualizado correctamente" || echo "la actualizacion del apt cache fallo, es posible que recibas paquetes obsoletos"

#Asignar archivos al directorio


os.makedirs("/workspaces/codespaces-jupyter/Minecraft-server", exist_ok=True)

os.chdir("/workspaces/codespaces-jupyter/Minecraft-server")

!ls #lista los archivos del directorio para verificarlos



#Importacion del archivo de configuracion


config_path = "/workspaces/codespaces-jupyter/Minecraft-server/colabconfig.json"

if os.path.isfile(config_path):

    with open(config_path) as config_file:

        colabconfig = json.load(config_file)

else:

    colabconfig = {"server_type": "generic"} # Usar la configura default si no existe

    with open(config_path, 'w') as new_config_file:

        json.dump(colabconfig, new_config_file)

# Instalar OpenJDK

version = colabconfig["server_version"]

# Ejecuta un comando en la terminal

def ejecutar_comando(comando):
    proceso = subprocess.run(comando, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    # Comprueba si el comando se ejecutó con éxito
    if proceso.returncode == 0:
        print("OpenJDK 17 instalado correctamente")
    else:
        print("Fallo al instalar OpenJDK 17")

# Purga versiones anteriores de OpenJDK
ejecutar_comando('sudo apt-get purge openjdk* -y')

# Instala OpenJDK 17
ejecutar_comando('sudo apt-get install openjdk-17-jre-headless -y')

# Chequeo de version de java

java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'

#Lista de posibles archivos .jar

jar_list = {'paper': 'paper.jar', 'fabric': 'fabric-server-launch.jar', 'generic': 'server.jar', 'forge': 'forge.jar', 'vanilla': 'vanilla.jar', 'snapshot': 'snapshot.jar', 'mohist': "mohist.jar", 'catserver': "catserver.jar", 'purpur': "purpur.jar"}

jar_name = jar_list[colabconfig["server_type"]]

# Argumentos de java.

if colabconfig["server_type"] == "paper":

    server_flags = "-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true"

else:

    server_flags = "" # No se necesitan banderas para servidores que no sean de paper

#Argumentos JVM

memory_allocation = "-Xms10G -Xmx10G"



# Elige el servicio de tunelacion que desees

# Opciones disponibles: ngrok

tunnel_service = "ngrok"

print("Usando", tunnel_service)


#Condicional de tunelacion del servidor



if tunnel_service == "ngrok":

    !pip -q install pyngrok  #Aqui se instala el ngrok en el servidor

    from pyngrok import conf, ngrok



    # Obten tu token de ngrok

    print("Consigue tu authtoken de https://dashboard.ngrok.com/auth") #Consigue aqui tu token de ngrok, sera dinamico, si quieres que sea estatico, necesitaras la version premiun de ngrok

    import getpass



    # v - - - - - - - TOKEN - - - - - - - v



    authtoken = "2gil3VJs7j9SUr2v2NDGbEZfLET_2YjrwUQxM9rCHiAaeou7i" # <---- TOKEN NGROK (PON AQUÍ TU TOKEN [TIENE QUE ESTAR ENTRE LAS "COMILLAS"])

    ! ngrok authtoken $authtoken # logeo en ngrok



    #Establecimiento de region de ngrok



    #v - - - - - - - REGIONES DISPONIBLES - - - - - - - v

    # ap - Asia/Pacifico (Singapore)

    # au - Australia (Sydney)

    # eu - Europa (Frankfurt - Alemania)

    # in - India (Mumbai)

    # jp - Japon (Tokyo)

    # sa - America del sur (São Paulo - Brasil)

    # us - Estados unidos (Ohio)

    conf.get_default().region = 'sa' # <--- Cambia esto por la region que quieras



    # Conectar a ngrok

    url = ngrok.connect(25565, 'tcp')

    print('La IP de tu servidor es ===>   ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', ''))

    #Si obtienes el servicio premiun de ngrok borra las dos lineas de codigo de arriba y quitale el # a estas a continuacion:

    #subdominio = "mi-subdominio"

    #url = ngrok.connect(25565, 'tcp', subdomain=subdominio)

    #print('La IP de tu servidor es ' + url.replace('tcp://', ''))

elif tunnel_service == "playit":

    ! curl -SsL https://playit-cloud.github.io/ppa/key.gpg | sudo apt-key add -
    ! sudo curl -SsL -o /etc/apt/sources.list.d/playit-cloud.list https://playit-cloud.github.io/ppa/playit-cloud.list
    ! sudo apt update &>/dev/null && sudo apt install playit &>/dev/null && echo "Playit.gg installed" || echo "Failed to install playit"
    print('Starting server...')
    ! playit & java $memory_allocation $server_flags -jar $jar_name nogui

print('Iniciando servidor...')

def get_users():
    url = 'https://jsonplaceholder.typicode.com/users'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            print("Lista de usuarios:")
            for user in data:
                print(f"Nombre: {user['name']}, Email: {user['email']}")
        else:
            print(f"Error al hacer la solicitud. Código de estado: {response.status_code}")
    except requests.RequestException as e:
        print(f"Error de conexión: {e}")

# Función para ejecutar get_users() cada 25 minutos
def repeat_get_users():
    while True:
        get_users()
        time.sleep(600)  # Espera 10 minutos

# Inicia un hilo para ejecutar repeat_get_users() en segundo plano
user_thread = threading.Thread(target=repeat_get_users)
user_thread.start()

if colabconfig["server_type"] == "forge":

    version = colabconfig["server_version"]
    
    forgefiles = f"/workspaces/codespaces-jupyter/Minecraft-server/libraries/net/minecraftforge/forge/"
    
    forgepreversion = os.listdir(forgefiles)
    
    if forgepreversion:
    
        forgeversion = forgepreversion[0]
        forgeversionchecked = forgeversion.replace(".jar", "")
        print("La versión de Forge es:", forgeversionchecked)
    
    else:
    
        print("No se encontraron archivos en el directorio Forge.")
    
    pathtoforge = glob.glob(f"/workspaces/codespaces-jupyter/Minecraft-server/libraries/net/minecraftforge/forge/{forgeversionchecked}/unix_args.txt")

    if pathtoforge: # Checa si la lista no esta vacia

        path = pathtoforge[0] # Obtiene la primera ruta de la lista

        print(path)

        !java @user_jvm_args.txt "@{path}" "$@"

    else:

        print("No se encontro el archivo unix_args.txt.")

#Si el servidor es de tipo Vanilla o paper o mohist o catserver o purpur

else:
    #Ejecuta el archivo jar del servidor
    
    !java $memory_allocation $server_flags -jar $jar_name nogui   

# Ciclo para realizar las solicitudes a JSONPlaceholder cada minuto

while True:
    time.sleep(60)  # Espera 1 minuto

195.57s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


UnboundLocalError: local variable 'child' referenced before assignment